In [ ]:
!pip install transformers
!pip install eli5

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/My\ Drive/Colab\ Notebooks/GPT

Mounted at /content/gdrive
/content/gdrive/My Drive/Colab Notebooks/GPT


In [ ]:
import time
import torch
import itertools
import math
import numpy as np
import random
import os, re
from os import walk
import pandas as pd
import transformers
from transformers import GPT2LMHeadModel, GPT2TokenizerFast
from transformers import pipeline
from transformers import T5Tokenizer
from transformers import AutoTokenizer, BartForConditionalGeneration
from collections import OrderedDict
from scipy.stats import norm
from difflib import SequenceMatcher
from multiprocessing.pool import ThreadPool
from sklearn.metrics import accuracy_score
from nltk import tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# Import generic wrappers
from transformers import AutoModel, AutoTokenizer , AutoModelForSequenceClassification, pipeline
import numpy as np
import pandas as pd

# Define the model repo
model_name = "roberta-base-openai-detector" 

# Download pytorch model
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of the model checkpoint at roberta-base-openai-detector were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
class gen_args:
    # folder path
    root_path = r'./our_dataset'
    chatgpt_path = root_path + '/chatgpt'    
    human_path = root_path + '/human'
    question_path = root_path + r'/questions.csv'
args_add=gen_args()

In [ ]:
questions=pd.read_csv(args_add.question_path, index_col=0, encoding='cp1252')
# list to store files name
chatgpt_files = []
human_files = []
for (dir_path, dir_names, file_names) in walk(args_add.chatgpt_path):
    chatgpt_files.extend(file_names)

# for (dir_path, dir_names, file_names) in walk(args_add.human_path):
#     human_files.extend(file_names)

In [ ]:
questions=pd.read_csv(args_add.question_path, index_col=0, encoding='cp1252')
# list to store files name
chatgpt_files = []
human_files = []
for (dir_path, dir_names, file_names) in walk(args_add.chatgpt_path):
    chatgpt_files.extend(file_names)

for (dir_path, dir_names, file_names) in walk(args_add.human_path):
    human_files.extend(file_names)
chatgpt_files_path = [os.path.join(args_add.chatgpt_path, f) for f in chatgpt_files]
human_files_path = [os.path.join(args_add.human_path, f) for f in human_files]
human_files_no = [re.split("\.",x)[0] for x in human_files]
chatgpt_files_no = [re.split("\.",x)[0] for x in chatgpt_files]

df_chatgpt=pd.DataFrame(columns=['sr_no', 'text'])
for i, f in zip(chatgpt_files_no, chatgpt_files_path):
  fileObject = open(f, "r")
  data = fileObject.read()
  # print(i)
  # print("------")
  # print(data)
  df_chatgpt = df_chatgpt.append({"sr_no":i, "text": data}, ignore_index=True)

df_human=pd.DataFrame(columns=['sr_no', 'text'])
for i, f in zip(human_files_no, human_files_path):
  fileObject = open(f, "r")
  data = fileObject.read()
  # print(i)
  # print("------")
  # print(data)
  df_human = df_human.append({"sr_no":i, "text": data}, ignore_index=True)

df_human['source']='human'
df_chatgpt['source']='chatgpt'
df_combined=pd.concat([df_human, df_chatgpt], axis=0)
df_combined.to_csv("our_dataset_chatgpt_human_combined.csv")
df_combined.reset_index(drop =True, inplace=True)
df_combined['text']= df_combined.text.map(lambda x: x.replace('\r', ' ').replace('\n', ' ').replace('  ', ' '))

In [ ]:
df_delta=pd.DataFrame(columns=['label', 'score'])
for txt in df_combined['text']:
  pred_pipeline  = pipeline("text-classification", model="roberta-base-openai-detector")
  # print(len(my_pipeline.tokenizer.tokenize(input_text)))
  df_delta=df_delta.append(pred_pipeline(txt, truncation=True))

In [ ]:
print(df_delta.shape)
print(df_combined.shape)

(366, 2)
(366, 3)


In [ ]:
df_combined.reset_index(drop=True, inplace=True)
df_delta.reset_index(drop=True, inplace=True)
df_final= pd.concat([df_combined, df_delta], axis=1)
df_final['label'] = df_final['label'].map({'Fake':'chatgpt', 'Real':'human'})

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
print(classification_report(df_combined['source'], df_final['label']))
print(confusion_matrix(df_combined['source'], df_final['label']))

              precision    recall  f1-score   support

     chatgpt       0.94      0.97      0.95       183
       human       0.97      0.93      0.95       183

    accuracy                           0.95       366
   macro avg       0.95      0.95      0.95       366
weighted avg       0.95      0.95      0.95       366

[[178   5]
 [ 12 171]]


In [ ]:
df_final.to_csv("our_dataset_roberta-base-openai-detector_predictions.csv")